# Backpack

In [1]:
from transformers import AutoTokenizer
from transformers import BertTokenizer
import torch
import os
import numpy as np
import torch.nn.functional as F
from scipy.stats import spearmanr

In [2]:
model = torch.load("/home/hamvir/NLP/Project/dataset/bert/results_1/model.pt", map_location=torch.device('cpu'))  # You can specify the device (e.g., 'cuda:0') if using GPU

In [ ]:
model.eval()

In [4]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [5]:
def simm(w1,w2):
    x1 = tokenizer(w1,return_tensors='pt',add_special_tokens=False)['input_ids']
    x1 = model.backpack.word_embeddings(x1)
    x1 = model.backpack.sense_network(x1).detach()
    x1 = x1[:,:,0,:]
    x1 = x1.squeeze()
    x1 = F.normalize(x1,p=2,dim=1)
    x2 = tokenizer(w2,return_tensors='pt',add_special_tokens=False)['input_ids']
    x2 = model.backpack.word_embeddings(x2)
    x2 = model.backpack.sense_network(x2).detach()
    x2 = x2[:,:,0,:]
    x2 = x2.squeeze()
    x2 = F.normalize(x2,p=2,dim=1)
    sim_list = torch.sum(torch.multiply(x1,x2),dim=1)
    #print(sim_list)
    return(sim_list.min())

## SIMVERB

In [6]:
word1 = []
word2 = []  # add space
h_score = []
m_score =[]

In [7]:
with open('/home/hamvir/NLP/Project/dataset/Hamvir/gpt_2_trained/SimVerb-3500.txt', 'r') as file:
    # Iterate through each line
    for line in file:
        # Split the line into columns
        columns = line.strip().split('\t')

        # Extract word1, word2, and the score
        word1.append(" "+ columns[0])
        word2.append(" "+ columns[1])
        h_score.append(float(columns[3]))  # Assuming the score is a floating-point number


In [8]:
for i in range(len(word1)):
    m_score.append(simm(word1[i],word2[i]))

In [9]:
spearman_corr, _ = spearmanr(h_score, m_score)
spearman_corr

0.05818864807522393

## SIMMLEX

In [10]:
word1 = []
word2 = []  # add space
h_score = []
m_score =[]

In [11]:
with open('/home/hamvir/NLP/Project/dataset/Hamvir/gpt_2_trained/SimLex-999.txt', 'r') as file:
    # Iterate through each line
    header = next(file)
    for line in file:
        # Split the line into columns
        columns = line.strip().split('\t')

        # Extract word1, word2, and the score
        word1.append(" "+ columns[0])
        word2.append(" "+ columns[1])
        h_score.append(float(columns[3]))  # Assuming the score is a floating-point number


In [12]:
for i in range(len(word1)):
    m_score.append(simm(word1[i],word2[i]))

In [13]:
spearman_corr, _ = spearmanr(h_score, m_score)
spearman_corr

0.10843808534891415